In [68]:
import pandas as pd
import numpy as np
import os
import geopy.distance
import json
import requests

# from googlemaps import Client as GoogleMaps
from geopy.geocoders import Nominatim
from sklearn import preprocessing
from collections import namedtuple

In [42]:
data_path = "/Users/vincentchu/Dev/oakland-crime-prediction/data/"

data = os.path.join(os.getcwd(),data_path + 'crimePublicData.csv')
crime_data = pd.read_csv(data)#.to_sparse()

In [43]:
print "Shape of crime_data: ", crime_data.shape
crime_data.describe()

Shape of crime_data:  (9109, 8)


,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State
count,9109,9104,9109,9109,8992,9109,9109,9109
unique,14,6793,9095,240,63,4785,1,1
top,THEFT/LARCENY,2018-01-01 00:00:00,18-002107,BURGLARY-AUTO,04X,400 7TH ST,Oakland,CA
freq,2816,12,2,1597,489,100,9109,9109


In [44]:
crime_data["CrimeType"].unique()

array(['MOTOR VEHICLE THEFT', 'ROBBERY', 'VANDALISM', 'THEFT/LARCENY',
       'FRAUD', 'BURGLARY', 'ASSAULT', 'SEX CRIMES', 'DUI',
       'DISTURBING THE PEACE', 'DRUGS/ALCOHOL VIOLATIONS', 'WEAPONS',
       'ARSON', 'HOMICIDE'], dtype=object)

In [45]:
crime_data.columns

Index([u'CrimeType', u'DateTime', u'CaseNumber', u'Description', u'PoliceBeat',
       u'Address', u'City', u'State'],
      dtype='object')

In [47]:
crime_data["CrimeType"].value_counts()

THEFT/LARCENY               2816
ASSAULT                     1821
MOTOR VEHICLE THEFT         1412
VANDALISM                    994
BURGLARY                     540
ROBBERY                      540
DRUGS/ALCOHOL VIOLATIONS     322
DISTURBING THE PEACE         225
FRAUD                        110
WEAPONS                       95
SEX CRIMES                    87
DUI                           73
ARSON                         54
HOMICIDE                      20
Name: CrimeType, dtype: int64

In [49]:
crime_data.head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State
0,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA
1,ROBBERY,2030-01-11 19:00:00,14-023460,"ROBBERY - STRONG ARM (HANDS, FISTS, FEET, ETC.)",23X,3600 BLK OF INTERNATIONAL BL,Oakland,CA
2,VANDALISM,2040-02-01 21:40:00,14-035766,"VANDALISM [UNDER $1,000] (AMENDED)",09X,600 FAIRMOUNT AV,Oakland,CA
3,THEFT/LARCENY,2050-01-03 13:55:00,14-038177,BURGLARY-AUTO,08X,TELEGRAPH AV & W MACARTHUR BLVD,Oakland,CA
4,FRAUD,2018-02-07 22:15:00,18-902981,USE ANOTHER'S PERSONAL IDENTIFICATION TO OBTAI...,13Z,00 VISTA DEL MAR CT,Oakland,CA


In [50]:
# Adding columns for Lat, Long and Zip
crime_data['Lat'] = 0.0
crime_data['Long'] = 0.0
crime_data['Zip'] = 0

crime_data_copy = crime_data.copy()

In [73]:
# Default API key: replace with your own api key
google_api_key = ""

# Function to call Google's geocode api based on an address
# and returns a dictionary in json format
def get_geocode_json(address, api_key):
    geo_s ='https://maps.googleapis.com/maps/api/geocode/json'
    param = {'address': address, 'key': google_api_key }    
    response = requests.get(geo_s, params=param)
    result_dict = response.json()
    return result_dict    

In [75]:
# Get Lat/Long and Zip for all addresses in the current set of crime data

for i in range(crime_data.shape[0]):
    print "Iteration i = " + str(i)
    
    address = crime_data.at[crime_data.index[i], 'Address'] + ", " + crime_data.at[crime_data.index[i], 'City'] + ", " + crime_data.at[crime_data.index[i], 'State']
    result_dict = get_geocode_json(address, google_api_key)

    try:
        crime_data.at[crime_data.index[i], 'Lat'] = result_dict['results'][0]['geometry']['location']['lat']
    except:
        print "i = " + str(i) + ": Cannot find Latitude"
    
    try:
        crime_data.at[crime_data.index[i], 'Long'] = result_dict['results'][0]['geometry']['location']['lng']
    except:
        print "i = " + str(i) + ": Cannot find Longitude"
        
    try:    
        for j in range(len(result_dict['results'][0]['address_components'])):
            if result_dict['results'][0]['address_components'][j]['types'][0] == 'postal_code':
                crime_data.at[crime_data.index[i], 'Zip'] = result_dict['results'][0]['address_components'][j]['long_name']
    except:
        print "i = " + str(i) + ": Cannot find Address Components"

Iteration i = 2401
Iteration i = 2402
Iteration i = 2403
Iteration i = 2404
Iteration i = 2405
Iteration i = 2406
Iteration i = 2407
Iteration i = 2408
Iteration i = 2409
Iteration i = 2410
Iteration i = 2411
Iteration i = 2412
Iteration i = 2413
Iteration i = 2414
Iteration i = 2415
Iteration i = 2416
Iteration i = 2417
Iteration i = 2418
Iteration i = 2419
Iteration i = 2420
Iteration i = 2421
Iteration i = 2422
Iteration i = 2423
Iteration i = 2424
Iteration i = 2425
Iteration i = 2426
Iteration i = 2427
Iteration i = 2428
Iteration i = 2429
Iteration i = 2430
Iteration i = 2431
Iteration i = 2432
Iteration i = 2433
Iteration i = 2434
Iteration i = 2435
Iteration i = 2436
Iteration i = 2437
Iteration i = 2438
Iteration i = 2439
Iteration i = 2440
Iteration i = 2441
Iteration i = 2442
Iteration i = 2443
Iteration i = 2444
Iteration i = 2445
Iteration i = 2446
Iteration i = 2447
Iteration i = 2448
Iteration i = 2449
Iteration i = 2450
Iteration i = 2451
Iteration i = 2452
Iteration i 

Iteration i = 2829
Iteration i = 2830
Iteration i = 2831
Iteration i = 2832
Iteration i = 2833
Iteration i = 2834
Iteration i = 2835
Iteration i = 2836
Iteration i = 2837
Iteration i = 2838
Iteration i = 2839
Iteration i = 2840
Iteration i = 2841
Iteration i = 2842
Iteration i = 2843
Iteration i = 2844
Iteration i = 2845
Iteration i = 2846
Iteration i = 2847
Iteration i = 2848
Iteration i = 2849
Iteration i = 2850
Iteration i = 2851
Iteration i = 2852
Iteration i = 2853
Iteration i = 2854
Iteration i = 2855
Iteration i = 2856
Iteration i = 2857
Iteration i = 2858
Iteration i = 2859
Iteration i = 2860
Iteration i = 2861
Iteration i = 2862
Iteration i = 2863
Iteration i = 2864
Iteration i = 2865
Iteration i = 2866
Iteration i = 2867
Iteration i = 2868
Iteration i = 2869
Iteration i = 2870
Iteration i = 2871
Iteration i = 2872
Iteration i = 2873
Iteration i = 2874
Iteration i = 2875
Iteration i = 2876
Iteration i = 2877
Iteration i = 2878
Iteration i = 2879
Iteration i = 2880
Iteration i 

Iteration i = 3256
Iteration i = 3257
Iteration i = 3258
Iteration i = 3259
Iteration i = 3260
Iteration i = 3261
Iteration i = 3262
Iteration i = 3263
Iteration i = 3264
Iteration i = 3265
Iteration i = 3266
Iteration i = 3267
Iteration i = 3268
Iteration i = 3269
Iteration i = 3270
Iteration i = 3271
Iteration i = 3272
Iteration i = 3273
Iteration i = 3274
Iteration i = 3275
Iteration i = 3276
Iteration i = 3277
Iteration i = 3278
Iteration i = 3279
Iteration i = 3280
Iteration i = 3281
Iteration i = 3282
Iteration i = 3283
Iteration i = 3284
Iteration i = 3285
Iteration i = 3286
Iteration i = 3287
Iteration i = 3288
Iteration i = 3289
Iteration i = 3290
Iteration i = 3291
Iteration i = 3292
Iteration i = 3293
Iteration i = 3294
Iteration i = 3295
Iteration i = 3296
Iteration i = 3297
Iteration i = 3298
Iteration i = 3299
Iteration i = 3300
Iteration i = 3301
Iteration i = 3302
Iteration i = 3303
Iteration i = 3304
Iteration i = 3305
Iteration i = 3306
Iteration i = 3307
Iteration i 

Iteration i = 3670
Iteration i = 3671
Iteration i = 3672
Iteration i = 3673
Iteration i = 3674
Iteration i = 3675
Iteration i = 3676
Iteration i = 3677
Iteration i = 3678
Iteration i = 3679
Iteration i = 3680
Iteration i = 3681
Iteration i = 3682
Iteration i = 3683
Iteration i = 3684
Iteration i = 3685
Iteration i = 3686
Iteration i = 3687
Iteration i = 3688
Iteration i = 3689
Iteration i = 3690
Iteration i = 3691
Iteration i = 3692
Iteration i = 3693
Iteration i = 3694
Iteration i = 3695
Iteration i = 3696
Iteration i = 3697
Iteration i = 3698
Iteration i = 3699
Iteration i = 3700
Iteration i = 3701
Iteration i = 3702
Iteration i = 3703
Iteration i = 3704
Iteration i = 3705
Iteration i = 3706
Iteration i = 3707
Iteration i = 3708
Iteration i = 3709
Iteration i = 3710
Iteration i = 3711
Iteration i = 3712
Iteration i = 3713
Iteration i = 3714
Iteration i = 3715
Iteration i = 3716
Iteration i = 3717
Iteration i = 3718
Iteration i = 3719
Iteration i = 3720
Iteration i = 3721
Iteration i 

Iteration i = 4103
Iteration i = 4104
Iteration i = 4105
Iteration i = 4106
Iteration i = 4107
Iteration i = 4108
Iteration i = 4109
Iteration i = 4110
Iteration i = 4111
Iteration i = 4112
Iteration i = 4113
Iteration i = 4114
Iteration i = 4115
Iteration i = 4116
Iteration i = 4117
Iteration i = 4118
Iteration i = 4119
Iteration i = 4120
Iteration i = 4121
Iteration i = 4122
Iteration i = 4123
Iteration i = 4124
Iteration i = 4125
Iteration i = 4126
Iteration i = 4127
Iteration i = 4128
Iteration i = 4129
Iteration i = 4130
Iteration i = 4131
Iteration i = 4132
Iteration i = 4133
Iteration i = 4134
Iteration i = 4135
Iteration i = 4136
Iteration i = 4137
Iteration i = 4138
Iteration i = 4139
Iteration i = 4140
Iteration i = 4141
Iteration i = 4142
Iteration i = 4143
Iteration i = 4144
Iteration i = 4145
Iteration i = 4146
Iteration i = 4147
Iteration i = 4148
Iteration i = 4149
Iteration i = 4150
Iteration i = 4151
Iteration i = 4152
Iteration i = 4153
Iteration i = 4154
Iteration i 

...

Iteration i = 7061
Iteration i = 7062
Iteration i = 7063
Iteration i = 7064
Iteration i = 7065
Iteration i = 7066
Iteration i = 7067
Iteration i = 7068
Iteration i = 7069
Iteration i = 7070
Iteration i = 7071
Iteration i = 7072
Iteration i = 7073
Iteration i = 7074
Iteration i = 7075
Iteration i = 7076
Iteration i = 7077
Iteration i = 7078
Iteration i = 7079
Iteration i = 7080
Iteration i = 7081
Iteration i = 7082
Iteration i = 7083
Iteration i = 7084
Iteration i = 7085
Iteration i = 7086
Iteration i = 7087
Iteration i = 7088
Iteration i = 7089
Iteration i = 7090
Iteration i = 7091
Iteration i = 7092
Iteration i = 7093
Iteration i = 7094
Iteration i = 7095
Iteration i = 7096
Iteration i = 7097
Iteration i = 7098
Iteration i = 7099
Iteration i = 7100
Iteration i = 7101
Iteration i = 7102
Iteration i = 7103
Iteration i = 7104
Iteration i = 7105
Iteration i = 7106
Iteration i = 7107
Iteration i = 7108
Iteration i = 7109
Iteration i = 7110
i = 7110: Cannot find Address Components
Iteration

Iteration i = 7486
Iteration i = 7487
Iteration i = 7488
Iteration i = 7489
Iteration i = 7490
Iteration i = 7491
Iteration i = 7492
Iteration i = 7493
Iteration i = 7494
Iteration i = 7495
Iteration i = 7496
Iteration i = 7497
Iteration i = 7498
Iteration i = 7499
Iteration i = 7500
Iteration i = 7501
Iteration i = 7502
Iteration i = 7503
Iteration i = 7504
Iteration i = 7505
Iteration i = 7506
Iteration i = 7507
Iteration i = 7508
Iteration i = 7509
Iteration i = 7510
Iteration i = 7511
Iteration i = 7512
Iteration i = 7513
Iteration i = 7514
Iteration i = 7515
Iteration i = 7516
Iteration i = 7517
Iteration i = 7518
Iteration i = 7519
Iteration i = 7520
Iteration i = 7521
Iteration i = 7522
Iteration i = 7523
Iteration i = 7524
Iteration i = 7525
Iteration i = 7526
Iteration i = 7527
Iteration i = 7528
Iteration i = 7529
Iteration i = 7530
Iteration i = 7531
Iteration i = 7532
Iteration i = 7533
Iteration i = 7534
Iteration i = 7535
Iteration i = 7536
Iteration i = 7537
Iteration i 

Iteration i = 7912
Iteration i = 7913
Iteration i = 7914
Iteration i = 7915
Iteration i = 7916
Iteration i = 7917
Iteration i = 7918
Iteration i = 7919
Iteration i = 7920
Iteration i = 7921
Iteration i = 7922
Iteration i = 7923
Iteration i = 7924
Iteration i = 7925
Iteration i = 7926
Iteration i = 7927
Iteration i = 7928
Iteration i = 7929
Iteration i = 7930
Iteration i = 7931
Iteration i = 7932
Iteration i = 7933
Iteration i = 7934
Iteration i = 7935
Iteration i = 7936
Iteration i = 7937
Iteration i = 7938
Iteration i = 7939
Iteration i = 7940
Iteration i = 7941
Iteration i = 7942
Iteration i = 7943
Iteration i = 7944
Iteration i = 7945
Iteration i = 7946
Iteration i = 7947
Iteration i = 7948
i = 7948: Cannot find Latitude
i = 7948: Cannot find Longitude
i = 7948: Cannot find Address Components
Iteration i = 7949
Iteration i = 7950
Iteration i = 7951
Iteration i = 7952
Iteration i = 7953
Iteration i = 7954
Iteration i = 7955
Iteration i = 7956
Iteration i = 7957
Iteration i = 7958
Ite

Iteration i = 8327
Iteration i = 8328
Iteration i = 8329
Iteration i = 8330
Iteration i = 8331
Iteration i = 8332
Iteration i = 8333
Iteration i = 8334
Iteration i = 8335
Iteration i = 8336
Iteration i = 8337
Iteration i = 8338
Iteration i = 8339
Iteration i = 8340
Iteration i = 8341
Iteration i = 8342
Iteration i = 8343
Iteration i = 8344
Iteration i = 8345
Iteration i = 8346
Iteration i = 8347
Iteration i = 8348
Iteration i = 8349
Iteration i = 8350
Iteration i = 8351
Iteration i = 8352
Iteration i = 8353
Iteration i = 8354
Iteration i = 8355
Iteration i = 8356
Iteration i = 8357
Iteration i = 8358
Iteration i = 8359
Iteration i = 8360
Iteration i = 8361
Iteration i = 8362
Iteration i = 8363
Iteration i = 8364
Iteration i = 8365
Iteration i = 8366
Iteration i = 8367
Iteration i = 8368
Iteration i = 8369
Iteration i = 8370
Iteration i = 8371
Iteration i = 8372
Iteration i = 8373
Iteration i = 8374
Iteration i = 8375
Iteration i = 8376
Iteration i = 8377
Iteration i = 8378
Iteration i 

Iteration i = 8748
Iteration i = 8749
Iteration i = 8750
Iteration i = 8751
Iteration i = 8752
Iteration i = 8753
Iteration i = 8754
Iteration i = 8755
Iteration i = 8756
Iteration i = 8757
Iteration i = 8758
Iteration i = 8759
Iteration i = 8760
Iteration i = 8761
Iteration i = 8762
Iteration i = 8763
Iteration i = 8764
Iteration i = 8765
Iteration i = 8766
Iteration i = 8767
Iteration i = 8768
Iteration i = 8769
Iteration i = 8770
Iteration i = 8771
Iteration i = 8772
Iteration i = 8773
Iteration i = 8774
Iteration i = 8775
Iteration i = 8776
Iteration i = 8777
Iteration i = 8778
Iteration i = 8779
Iteration i = 8780
Iteration i = 8781
Iteration i = 8782
Iteration i = 8783
Iteration i = 8784
Iteration i = 8785
Iteration i = 8786
Iteration i = 8787
Iteration i = 8788
Iteration i = 8789
Iteration i = 8790
Iteration i = 8791
Iteration i = 8792
Iteration i = 8793
Iteration i = 8794
Iteration i = 8795
Iteration i = 8796
Iteration i = 8797
Iteration i = 8798
Iteration i = 8799
Iteration i 

In [76]:
# write crime_data to csv
crime_data.to_csv('crimePublicData_transformed.csv')

# Examine head of the data frame for validation
crime_data.head()

,CrimeType,DateTime,CaseNumber,Description,PoliceBeat,Address,City,State,Lat,Long,Zip
0,MOTOR VEHICLE THEFT,2104-08-08 00:00:00,14-040258,VEHICLE THEFT - AUTO,02X,12TH ST & MANDELA PKWY,Oakland,CA,37.809085,-122.291942,94607
1,ROBBERY,2030-01-11 19:00:00,14-023460,"ROBBERY - STRONG ARM (HANDS, FISTS, FEET, ETC.)",23X,3600 BLK OF INTERNATIONAL BL,Oakland,CA,37.775883,-122.221834,94601
2,VANDALISM,2040-02-01 21:40:00,14-035766,"VANDALISM [UNDER $1,000] (AMENDED)",09X,600 FAIRMOUNT AV,Oakland,CA,37.822005,-122.253938,94611
3,THEFT/LARCENY,2050-01-03 13:55:00,14-038177,BURGLARY-AUTO,08X,TELEGRAPH AV & W MACARTHUR BLVD,Oakland,CA,37.826355,-122.265215,94609
4,FRAUD,2018-02-07 22:15:00,18-902981,USE ANOTHER'S PERSONAL IDENTIFICATION TO OBTAI...,13Z,00 VISTA DEL MAR CT,Oakland,CA,37.837958,-122.205544,94611


In [77]:
# Print out Lat/Long and Zip of part of the data frame for validation
for i in range(0, 5):
    print "Iteration %d" % i
    print "(Lat, Long) = ", (crime_data.at[crime_data.index[i], 'Lat'], crime_data.at[crime_data.index[i], 'Long'])
    print "Zip = %d" % crime_data.at[crime_data.index[i], 'Zip']

Iteration 0
(Lat, Long) =  (37.80908470000001, -122.2919421)
Zip = 94607
Iteration 1
(Lat, Long) =  (37.7758827, -122.2218341)
Zip = 94601
Iteration 2
(Lat, Long) =  (37.8220053, -122.2539378)
Zip = 94611
Iteration 3
(Lat, Long) =  (37.8263547, -122.2652151)
Zip = 94609
Iteration 4
(Lat, Long) =  (37.8379578, -122.2055437)
Zip = 94611
